### Importing Libraries

In [5]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from imblearn.combine import SMOTEENN

import numpy as np 
import pandas as pd 
from io import StringIO
import seaborn as sns 
import matplotlib.ticker as mtick  
import matplotlib.pyplot as plt
%matplotlib inline
import io
from io import StringIO
import string
from collections import Counter

try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO






#### Reading csv

In [6]:

from google.colab import files
  
  
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['tel_churn.csv']))

df.head()

Saving tel_churn.csv to tel_churn (2).csv


,Unnamed: 0,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,0,29.85,29.85,0,1,0,0,1,1,...,0,0,1,0,1,0,0,0,0,0
1,1,0,56.95,1889.50,0,0,1,1,0,1,...,0,0,0,1,0,0,1,0,0,0
2,2,0,53.85,108.15,1,0,1,1,0,1,...,0,0,0,1,1,0,0,0,0,0
3,3,0,42.30,1840.75,0,0,1,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4,4,0,70.70,151.65,1,1,0,1,0,1,...,0,0,1,0,1,0,0,0,0,0


In [7]:
df.shape

(7032, 52)

In [9]:
df.columns

Index(['Unnamed: 0', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges',
       'Churn', 'gender_Female', 'gender_Male', 'Partner_No', 'Partner_Yes',
       'Dependents_No', 'Dependents_Yes', 'PhoneService_No',
       'PhoneService_Yes', 'MultipleLines_No',
       'MultipleLines_No phone service', 'MultipleLines_Yes',
       'InternetService_DSL', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_No',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'OnlineBackup_No', 'OnlineBackup_No internet service',
       'OnlineBackup_Yes', 'DeviceProtection_No',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'TechSupport_No', 'TechSupport_No internet service', 'TechSupport_Yes',
       'StreamingTV_No', 'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No', 'StreamingMovies_No internet service',
       'StreamingMovies_Yes', 'Contract_Month-to-month', 'Contract_One year',
       'Contract

In [10]:
#extra unnamed here
df=df.drop('Unnamed: 0',axis=1)

In [11]:
df.shape

(7032, 51)

In [ ]:
x=df.drop('Churn',axis=1)
x

In [ ]:
y=df['Churn']
y

##### Train Test Split

In [14]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

#### Decision Tree Classifier

In [15]:
model_dt=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [16]:
model_dt.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [17]:
y_pred=model_dt.predict(x_test)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [18]:
model_dt.score(x_test,y_test)

0.8017057569296375

In [19]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.83      0.92      0.87      1026
           1       0.68      0.50      0.58       381

    accuracy                           0.80      1407
   macro avg       0.76      0.71      0.72      1407
weighted avg       0.79      0.80      0.79      1407



###### As you can see that the accuracy is quite low, and as it's an imbalanced dataset, we shouldn't consider Accuracy as our metrics to measure the model, as Accuracy is cursed in imbalanced datasets.

###### Hence, we need to check recall, precision & f1 score for the minority class, and it's quite evident that the precision, recall & f1 score is too low for Class 1, i.e. churned customers.

###### Hence, moving ahead to call SMOTEENN (UpSampling + ENN)

smote: Synthetic Minority Oversampling Technique (SMOTE) 
a statistical technique for increasing the number of cases in your dataset in a balanced way.

In [21]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(x,y)

In [22]:
xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

In [23]:
model_dt_smote=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [24]:
model_dt_smote.fit(xr_train,yr_train)
yr_predict = model_dt_smote.predict(xr_test)
model_score_r = model_dt_smote.score(xr_test, yr_test)
print(model_score_r)
print(metrics.classification_report(yr_test, yr_predict))

0.927013422818792
              precision    recall  f1-score   support

           0       0.94      0.90      0.92       559
           1       0.91      0.95      0.93       633

    accuracy                           0.93      1192
   macro avg       0.93      0.93      0.93      1192
weighted avg       0.93      0.93      0.93      1192



In [25]:
print(metrics.confusion_matrix(yr_test, yr_predict))

[[503  56]
 [ 31 602]]


###### Now we can see quite better results, i.e. Accuracy: 92 %, and a very good recall, precision & f1 score for minority class.



#### Random Forest Classifier

In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
model_rf=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [29]:
model_rf.fit(x_train,y_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [30]:
y_pred=model_rf.predict(x_test)

In [31]:
model_rf.score(x_test,y_test)

0.8009950248756219

In [32]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.83      0.92      0.87      1026
           1       0.69      0.48      0.56       381

    accuracy                           0.80      1407
   macro avg       0.76      0.70      0.72      1407
weighted avg       0.79      0.80      0.79      1407



In [34]:
sm = SMOTEENN()
X_resampled1, y_resampled1 = sm.fit_resample(x,y)

In [35]:
xr_train1,xr_test1,yr_train1,yr_test1=train_test_split(X_resampled1, y_resampled1,test_size=0.2)

In [36]:
model_rf_smote=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [37]:
model_rf_smote.fit(xr_train1,yr_train1)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [38]:
yr_predict1 = model_rf_smote.predict(xr_test1)

In [39]:
model_score_r1 = model_rf_smote.score(xr_test1, yr_test1)

In [40]:
print(model_score_r1)
print(metrics.classification_report(yr_test1, yr_predict1))

0.9381443298969072
              precision    recall  f1-score   support

           0       0.95      0.91      0.93       516
           1       0.93      0.96      0.95       648

    accuracy                           0.94      1164
   macro avg       0.94      0.94      0.94      1164
weighted avg       0.94      0.94      0.94      1164



In [41]:
print(metrics.confusion_matrix(yr_test1, yr_predict1))

[[469  47]
 [ 25 623]]


###### With RF Classifier, also we are able to get quite good results, infact better than Decision Tree.



###### Now multiple classifiers

#### Pickling the model : the process of serializing an object in Python and saving it to a file

In [42]:
import pickle

In [43]:
filename = 'model.sav'

saving the state of a model :  model_rf_state in a file model.sav

In [44]:
pickle.dump(model_rf_smote, open(filename, 'wb'))

In [45]:
load_model = pickle.load(open(filename, 'rb'))

In [52]:
model_score_r1 = load_model.score(xr_test1, yr_test1)

In [53]:
model_score_r1

0.9381443298969072

##### RF Classifier with SMOTE => dumped in model=>  prepare API's=> to use this model from UI.